In [1]:
from osgeo import gdal, ogr, osr

from fiona.ogrext import Iterator, ItemsIterator, KeysIterator

from geopandas import GeoDataFrame

import plotly.figure_factory as ff

import pandas as pd

In [2]:
import plotly.io as pio
pio.renderers.default = 'notebook'

In [3]:
#input data gets defined here
lifeX_path = ('total_counties_02.csv') #your file organization

In [4]:
dfLifeX = pd.read_csv(lifeX_path)
dfLifeX

,Unnamed: 0,location_name,FIPS,year_id,val
0,0,Aitkin County,27001,1980,74.708551
1,1,Aitkin County,27001,1981,74.983337
2,2,Aitkin County,27001,1982,75.221320
3,3,Aitkin County,27001,1983,75.431234
4,4,Aitkin County,27001,1984,75.576746
...,...,...,...,...,...
109930,109930,Yellowstone County,30111,2010,78.798074
109931,109931,Yellowstone County,30111,2011,78.812881
109932,109932,Yellowstone County,30111,2012,78.854331
109933,109933,Yellowstone County,30111,2013,78.770020


In [5]:
closure_path = ('Closures-Database-for-Web_reduced.csv')

In [6]:
dfClosures = pd.read_csv(closure_path)
dfClosures.head()

,Zip,Closure Year
0,30428,2014
1,40311,2014
2,1247,2014
3,64776,2014
4,68781,2014


In [7]:
conversion_path = 'ZIP-COUNTY-FIPS_2014-03.csv'

In [8]:
dfConv = pd.read_csv(conversion_path)
dfConv.head()

,ZIP,COUNTYNAME,STATE,STCOUNTYFP,CLASSFP
0,36006,Autauga County,AL,1001,H1
1,36003,Autauga County,AL,1001,H1
2,36067,Autauga County,AL,1001,H1
3,36066,Autauga County,AL,1001,H1
4,36703,Autauga County,AL,1001,H1


In [9]:
dfClosures_with_FIPS = dfClosures.merge(dfConv, left_on='Zip', right_on='ZIP')
dfClosures_with_FIPS.head()

,Zip,Closure Year,ZIP,COUNTYNAME,STATE,STCOUNTYFP,CLASSFP
0,30428,2014,30428,Laurens County,GA,13175,H1
1,30428,2014,30428,Wheeler County,GA,13309,H1
2,40311,2014,40311,Bath County,KY,21011,H1
3,40311,2014,40311,Bourbon County,KY,21017,H1
4,40311,2014,40311,Nicholas County,KY,21181,H1


In [11]:
dfLifeX_with_Closures = dfClosures_with_FIPS.merge(dfLifeX, left_on='STCOUNTYFP', right_on='FIPS')
dfLifeX_with_Closures

,Zip,Closure Year,ZIP,COUNTYNAME,STATE,STCOUNTYFP,CLASSFP,Unnamed: 0,location_name,FIPS,year_id,val
0,30428,2014,30428,Laurens County,GA,13175,H1,76720,Laurens County,13175,1980,71.085651
1,30428,2014,30428,Laurens County,GA,13175,H1,76721,Laurens County,13175,1981,71.346549
2,30428,2014,30428,Laurens County,GA,13175,H1,76722,Laurens County,13175,1982,71.656628
3,30428,2014,30428,Laurens County,GA,13175,H1,76723,Laurens County,13175,1983,71.997129
4,30428,2014,30428,Laurens County,GA,13175,H1,76724,Laurens County,13175,1984,71.841958
...,...,...,...,...,...,...,...,...,...,...,...,...
5175,76444,2005,76444,Erath County,TX,48143,H1,21100,Erath County,48143,2010,78.419974
5176,76444,2005,76444,Erath County,TX,48143,H1,21101,Erath County,48143,2011,78.435775
5177,76444,2005,76444,Erath County,TX,48143,H1,21102,Erath County,48143,2012,78.540391
5178,76444,2005,76444,Erath County,TX,48143,H1,21103,Erath County,48143,2013,78.557192


In [14]:
#state_df = state_df[((state_df.measure_name == ‘Life expectancy’)
for year in range(2005, 2015):  # 2005-2015 eventually
    dfLifeXYear = dfLifeX_with_Closures[((dfLifeX_with_Closures.year_id == year))]
    fig = ff.create_choropleth(fips=dfLifeXYear['FIPS'], values=dfLifeXYear['val'], 
                               title=f'Life Expectancy ({year}) with Hospital Closures')
    fig.layout.template = None
    fig.update_layout(showlegend=False)
    #fig.show()
    fig.write_image(f"lifex_closure_{year}.png")